In [10]:
from cellpose import core, utils, io, models, metrics
from tifffile import imread, imwrite
from skimage.transform import resize
from pathlib import Path

In [34]:
import glob
import numpy as np
image_paths = "../..//pvc/scratch/interaction_cells/datasets/*.tif"
images = []
for image_path in glob.glob(image_paths):
    image = imread(image_path)[:, 0, ...] 
    #image = np.expand_dims(image,1)
    image =  resize(image, (image.shape[0] ,image.shape[1] // 2, image.shape[2] // 2), anti_aliasing=True)
    image = list([im for im in image])
    images.append(image)

In [39]:
cellpose_model = models.CellposeModel(gpu=True, model_type='cyto3')

In [42]:
masks = []
from tqdm import tqdm
from pathlib import Path
import glob
output_dir = "../..//pvc/scratch/SHARE/cancer_cell_masks"
Path(output_dir).mkdir(parents=True, exist_ok=True)
for image_path, image in tqdm(list(zip(glob.glob(image_paths), images))[10:]):
    mask, flows, styles = cellpose_model.eval(image,
        diameter=45, do_3D=False, channels=[0, 0], normalize=True, flow_threshold=0.6, cellprob_threshold=-1.0,batch_size=81
    )
    masks.append(mask)
        # Get original filename and create new path
    original_name = Path(image_path).name
    output_path = str(Path(output_dir) / original_name)
    
    # Save mask as tiff
    imwrite(output_path, mask)

100%|██████████| 4/4 [11:02<00:00, 165.62s/it]


In [37]:
# Save masks as tiff files with same structure as input
from pathlib import Path

output_dir = "../..//pvc/scratch/SHARE/cancer_cell_masks"
Path(output_dir).mkdir(parents=True, exist_ok=True)

for mask, image_path in zip(masks, glob.glob(image_paths)):
    # Get original filename and create new path
    original_name = Path(image_path).name
    output_path = str(Path(output_dir) / original_name)
    
    # Save mask as tiff
    imwrite(output_path, mask)

